In [1]:
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
from urllib.parse import urlparse
import requests
import json
import os


#### Receive articles from **aljazeera, cnn, foxnews**


In [2]:
def get_article_links(base_url, path_filter=None, keyword_filters=None, exclude_keywords=None, skip_base=False):
    article_links = []
    page = requests.get(base_url)
    web = BeautifulSoup(page.text, 'html.parser')

    for link in web.find_all('a', href=True):
        href = link['href']
        if href.startswith('/'):
            if path_filter and not href.startswith(path_filter):
                continue
            if keyword_filters and not any(keyword in href for keyword in keyword_filters):
                continue
            if exclude_keywords and any(exclude in href for exclude in exclude_keywords):
                continue
            if skip_base==True:
                href = 'https:' + href
                full_url = href
            else:
                full_url = base_url.rstrip('/') + href
            
            if full_url not in article_links:
                article_links.append(full_url)
    return article_links


all_article_urls = []

# Aljazeera
aljazeera_base_url = "https://www.aljazeera.com"
aljazeera_links = get_article_links(aljazeera_base_url, path_filter='/news/', keyword_filters=['/news/'])
all_article_urls.extend(aljazeera_links)

# CNN
cnn_base_url = "https://edition.cnn.com"
cnn_links = get_article_links(cnn_base_url, keyword_filters=['/politics/', '/business/'])
all_article_urls.extend(cnn_links)

# Fox News
foxnews_base_url = "https://www.foxnews.com"
foxnews_links = get_article_links(foxnews_base_url, keyword_filters=['/politics/'], skip_base=True)
all_article_urls.extend(foxnews_links)

for article in all_article_urls:
    print('article: ', article)

article_url = all_article_urls


article:  https://www.aljazeera.com/news/
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096854
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096798
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096744
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096773
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096777
article:  https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to

#### Receive articles from **NewYorkTimes**


In [3]:
URL1 = "https://www.nytimes.com/section/business/economy"
URL2 = "https://www.nytimes.com/section/technology"
URLS = [URL1, URL2]


def get_content_string(urls):
    content_strings = []

    for url in urls:
        page = requests.get(url)
        page_soup = BeautifulSoup(page.content, 'html.parser')
        containers = page_soup.find_all("script", {"type": "application/ld+json"})
        for container in containers:
            try:
                data = json.loads(container.string)
                content_strings.append(data)
            except json.JSONDecodeError:
                continue
    
    return content_strings



content_strings = get_content_string(URLS)
print('content_strings: ', content_strings)



def extract_urls(content_strings):
    urls = []

    for content in content_strings:
        if isinstance(content, dict) and 'mainEntity' in content:
            main_entity = content['mainEntity']
            if isinstance(main_entity, dict) and 'itemListElement' in main_entity:
                items = main_entity['itemListElement']
                for item in items:
                    if isinstance(item, dict) and '@type' in item and item['@type'] == 'ListItem' and 'url' in item:
                        urls.append(item['url'])

    return urls

urls = extract_urls(content_strings)

for article in urls:
    article_url.append(article)

content_strings:  [{'@context': 'https://schema.org', '@type': 'CollectionPage', 'description': 'The latest news on the economy, job market, real estate industry and workplace. Includes coverage of the Federal Reserve and government spending at the state and national level.', 'url': 'https://www.nytimes.com/section/business/economy', 'inLanguage': 'en-US', 'name': 'Economy', 'mainEntity': {'@context': 'https://schema.org', '@type': 'ItemList', 'itemListElement': [{'@context': 'https://schema.org', '@type': 'ListItem', 'url': 'https://www.nytimes.com/2024/08/02/us/politics/jd-vance-new-right-economics.html', 'position': 1}, {'@context': 'https://schema.org', '@type': 'ListItem', 'url': 'https://www.nytimes.com/2024/08/02/business/economy/stocks-drop-economy-jobs.html', 'position': 2}, {'@context': 'https://schema.org', '@type': 'ListItem', 'url': 'https://www.nytimes.com/2024/08/02/business/economy/jobs-report-fed.html', 'position': 3}, {'@context': 'https://schema.org', '@type': 'ListI

#### Summarize the articels and write in csv to create dataset

-> don't use liveblog, articles without date and author

In [4]:
def summarize_article(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    
    author_string = ", ".join(article.authors)
    date = article.publish_date
    date_string = date.strftime("%m/%d/%Y") if date else "No date available"
    summary = article.summary
    title = article.title
    lang = article.meta_lang  

    return summary, title, date_string, author_string, lang

csv_file = "articles.csv"

if os.path.isfile(csv_file):
    existing_df = pd.read_csv(csv_file)
    existing_urls = set(existing_df['URL'].tolist())
else:
    existing_urls = set()

data = []

for url in article_url:
    if url not in existing_urls:
        print("Processing article URL: " + str(url))
        summary, title, date, author, lang = summarize_article(url)
        if (date != "No date available" or author.strip()) and 'liveblog' not in url:  # Exclude articles with "No date available", no author, and "liveblog" in URL
            source = urlparse(url).netloc
            title_length = len(title)
            summary_length = len(summary)
            data.append([title, summary, date, author, source, url, title_length, summary_length, lang])
        else:
            print(f"Skipping article with no date, no author, or liveblog URL: {url}")
    else:
        print(f"Skipping duplicate article URL: {url}")

if data:
    new_df = pd.DataFrame(data, columns=["Title", "Summary", "Date", "Author", "Source", "URL", "Title Length", "Summary Length", "Language"])
    
    if not os.path.isfile(csv_file):
        new_df.to_csv(csv_file, index=False)
    else:
        new_df.to_csv(csv_file, mode='a', header=False, index=False)

    print("CSV file 'articles.csv' has been updated successfully.")
else:
    print("No new articles to add.")


Processing article URL: https://www.aljazeera.com/news/
Skipping article with no date, no author, or liveblog URL: https://www.aljazeera.com/news/
Processing article URL: https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests
Skipping article with no date, no author, or liveblog URL: https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests
Processing article URL: https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096854
Skipping article with no date, no author, or liveblog URL: https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096854
Processing article URL: https://www.aljazeera.com/news/liveblog/2024/8/5/bangladesh-protests-live-army-chief-to-address-nation-amid-new-protests?update=3096798
Skipping arti

In [5]:
csv_file = "articles.csv"
df = pd.read_csv(csv_file)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.width', None)  

df

,Title,Summary,Date,Author,Source,URL,Title Length,Summary Length,Language
0,"Kerala landslides death toll hits 151, many missing as rain hampers rescue","Hundreds of rescue workers search through mud and debris for a second day after multiple landslides hit a hilly district in India’s southern state.\nHundreds of rescue workers are searching through mud and debris from multiple landslides that have killed at least 151 people in southern India’s Kerala state, police said.\n“The landslides happened at the top and then affected the lower valley areas where people were staying.\nTelevision footage showed rescue workers making their way through mud and uprooted trees to reach those who had been stranded.\nNearly 500 people were killed around Kerala in 2018 during the worst flooding to hit the state in almost a century.",07/31/2024,NaN,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/india-kerala-landslides-death-toll-hits-151-many-missing-as-rain-hampers-rescue,74,667,en
1,World reacts to killing of Hamas political chief Haniyeh in Iran,"Hamas’s political chief was in Tehran when he was killed, with the Palestinian group that governs Gaza blaming Israel.\nHamas’s political leader Ismail Haniyeh has been killed in Iran’s capital Tehran.\nHis death has been confirmed in separate statements by the Palestinian group that runs the Gaza Strip and Iran’s elite Islamic Revolutionary Guard Corps.\nIsrael is on the verge of collapse, and its reactions reflect confusion and inability to achieve any of its goals.\nIranian Foreign Ministry spokesman Nasser Kanaani“Haniyeh’s martyrdom in Tehran will strengthen the deep and unbreakable bond between Tehran, Palestine, and the resistance,” Kanaani said, according to Iranian state media.",07/31/2024,NaN,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/reactions-to-the-killing-of-hamass-ismail-haniyeh,64,691,en
2,Why protest by ethnic Baloch has put Pakistan’s key port of Gwadar on edge,"Islamabad, Pakistan — Tensions are high in Pakistan’s port city of Gwadar in the southwestern Balochistan province where an ethnic Baloch group has been protesting for days, following the arrests of some of their members and deadly clashes with security forces.\nGwadar is Pakistan’s only deep-sea port on the Arabian Sea, and is a key route of the $60bn China-Pakistan Economic Corridor (CPEC).\nThe ethnic Baloch allege that the Pakistani state has neglected their community and exploited the province’s mineral resources.\nArrests and blockadesOn Sunday, the BYC called for a “Baloch Rajee Muchi”, or Baloch National Gathering, in Gwadar.\n“The paramilitary force opened fire on a convoy comprising hundreds of people, including women and children, who were travelling from Quetta to Gwadar,” BYC representative Bebarg Baloch told Al Jazeera.",07/31/2024,Abid Hussain,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/why-protest-by-ethnic-baloch-has-put-pakistans-key-port-of-gwadar-on-edge,74,841,en
3,‘Childless cat ladies’: Has JD Vance taken on 22 million US women?,"Trump had chosen Vance as his running mate ahead of the Republican National Convention in July.\nVance made the comment in a 2021 interview with former Fox News host Tucker Carlson.\nTransportation Secretary Buttigieg and his husband, Chasten Buttigieg, adopted twins in August 2021, the same year that Vance made his comment.\nVance’s statements are likely not going to impact the women who are die-hard Trump supporters, Walsh said.\nHowever, it “does not seem like” they are trying to appeal to women who do not typically vote Republican, she said.",07/30/2024,Sarah Shamim,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/30/childless-cat-ladies-has-jd-vance-taken-on-22-million-us-women,66,547,en
4,"Far-right protesters target Southport mosque, clash with UK police","Merseyside Police said that 22 officers were injured in Tuesday night’s violence, eight of them seriously.\nThe unrest began when several hundred people, most of them

In [6]:
filtered_df = df[(df['Date'] == 'No date available') & (df['Author'].isna())]
filtered_df

,Title,Summary,Date,Author,Source,URL,Title Length,Summary Length,Language


In [7]:
filtered_df = df[(df['Source'] == 'www.aljazeera.com')]
filtered_df

,Title,Summary,Date,Author,Source,URL,Title Length,Summary Length,Language
0,"Kerala landslides death toll hits 151, many missing as rain hampers rescue","Hundreds of rescue workers search through mud and debris for a second day after multiple landslides hit a hilly district in India’s southern state.\nHundreds of rescue workers are searching through mud and debris from multiple landslides that have killed at least 151 people in southern India’s Kerala state, police said.\n“The landslides happened at the top and then affected the lower valley areas where people were staying.\nTelevision footage showed rescue workers making their way through mud and uprooted trees to reach those who had been stranded.\nNearly 500 people were killed around Kerala in 2018 during the worst flooding to hit the state in almost a century.",07/31/2024,NaN,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/india-kerala-landslides-death-toll-hits-151-many-missing-as-rain-hampers-rescue,74,667,en
1,World reacts to killing of Hamas political chief Haniyeh in Iran,"Hamas’s political chief was in Tehran when he was killed, with the Palestinian group that governs Gaza blaming Israel.\nHamas’s political leader Ismail Haniyeh has been killed in Iran’s capital Tehran.\nHis death has been confirmed in separate statements by the Palestinian group that runs the Gaza Strip and Iran’s elite Islamic Revolutionary Guard Corps.\nIsrael is on the verge of collapse, and its reactions reflect confusion and inability to achieve any of its goals.\nIranian Foreign Ministry spokesman Nasser Kanaani“Haniyeh’s martyrdom in Tehran will strengthen the deep and unbreakable bond between Tehran, Palestine, and the resistance,” Kanaani said, according to Iranian state media.",07/31/2024,NaN,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/reactions-to-the-killing-of-hamass-ismail-haniyeh,64,691,en
2,Why protest by ethnic Baloch has put Pakistan’s key port of Gwadar on edge,"Islamabad, Pakistan — Tensions are high in Pakistan’s port city of Gwadar in the southwestern Balochistan province where an ethnic Baloch group has been protesting for days, following the arrests of some of their members and deadly clashes with security forces.\nGwadar is Pakistan’s only deep-sea port on the Arabian Sea, and is a key route of the $60bn China-Pakistan Economic Corridor (CPEC).\nThe ethnic Baloch allege that the Pakistani state has neglected their community and exploited the province’s mineral resources.\nArrests and blockadesOn Sunday, the BYC called for a “Baloch Rajee Muchi”, or Baloch National Gathering, in Gwadar.\n“The paramilitary force opened fire on a convoy comprising hundreds of people, including women and children, who were travelling from Quetta to Gwadar,” BYC representative Bebarg Baloch told Al Jazeera.",07/31/2024,Abid Hussain,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/31/why-protest-by-ethnic-baloch-has-put-pakistans-key-port-of-gwadar-on-edge,74,841,en
3,‘Childless cat ladies’: Has JD Vance taken on 22 million US women?,"Trump had chosen Vance as his running mate ahead of the Republican National Convention in July.\nVance made the comment in a 2021 interview with former Fox News host Tucker Carlson.\nTransportation Secretary Buttigieg and his husband, Chasten Buttigieg, adopted twins in August 2021, the same year that Vance made his comment.\nVance’s statements are likely not going to impact the women who are die-hard Trump supporters, Walsh said.\nHowever, it “does not seem like” they are trying to appeal to women who do not typically vote Republican, she said.",07/30/2024,Sarah Shamim,www.aljazeera.com,https://www.aljazeera.com/news/2024/7/30/childless-cat-ladies-has-jd-vance-taken-on-22-million-us-women,66,547,en
4,"Far-right protesters target Southport mosque, clash with UK police","Merseyside Police said that 22 officers were injured in Tuesday night’s violence, eight of them seriously.\nThe unrest began when several hundred people, most of them